# Todo (Issue)
- answer code 누적합 column 제작
- 그래프 데이터 만들기
- 해당 시험을 몇 번째 푸는지 여부 (11/21 완)
   - 해당 시험을 몇 번째 푸는지 여부 (11/21 완)
   - 시험 -> 문제, 태그로 변환도 가능하며 이전에 맞췄는지 여부도 포함 (11/21 완)
- 문제를 푸는데 걸린시간이 평균 시간보다 오래걸렸는지 여부 추가
   - 한 유저가 어떤 문제를 풀때 그 문제를 푸는데 걸린 시간이 그 문제의 평균 시간보다 적게 걸렸는지 혹은 오래 걸렸는지 여부를 column에 추가
   - 해당 문항을 맞춘 학생의 평균 시간과 틀린 학생의 평균 시간 2가지를 Feature로 주어 활용하는 방안도 있음
- 사용자의 정답률 column 제작 (11/21 완)
   - 누적 정답률, 최근 정답률 두 가지를 제작한다. 최근 정답률은 window size를 줘서 최근 N개의 문항에 대한 평균을 구할 수 있도록 (11/21 완, N개 문항은 모델 부분에서 적용)
- 문항, 시험지, 태그의 평균 정답률 (11/21 완)
   - [EDA] test에서 정답을 예측해야 되는 문제가 train에도 있는지 확인
- 문제를 푼 사용자와 사용자가 푼 문항을 MF나 SVD를 통해 user-item matrix 생성해 feature로 사용

# Todo (추가)
- 장원준 EDA (11/21 완)
   - 각 문항 번호 column 추가
   - 각 유저별 푼 카테고리 목록 추가
- 정준환 EDA
   - timestamp이용해 걸린 시간 계산
   - 오류 시간은 -1 처리, threshold 걸기(200초)
- 이수경 EDA
   - timestamp이용해 걸린 시간 계산(elapsed_time)
   - 한 유저가 동일한 시험지를 두 번 이상 풀었을 경우를 고려 못 해줌 => 고려 필요
   - 한 유저가 동일한 시험지를 연속해서 풀었을 수도 있음 => 확인 필요 => 288번 있음
   - 문제 번호는 랜덤이라는 것 주의해서 처리
   - 시간 이상치 처리(threshold 200초, 평균 처리 등)
   - 7863번 knowledgetag 제외 모두 고유한 category를 가진다는 것 유념
- 류명현 EDA
   - 시간대별 정답률 다르므로 timestamp를 나누자



In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [7]:
train = pd.read_csv('data/train_data.csv')
test = pd.read_csv('data/test_data.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [15]:
new_test = test.copy()
real_test = new_test[new_test['answerCode'] == -1]
new_test = new_test.drop(index = new_test[new_test['answerCode'] == -1].index, axis = 0)

In [16]:
new_train = train.copy()

In [17]:
test.shape # (260114, 6)
real_test.shape # (744, 6)
new_test.shape # (259370, 6)

new_train.shape # (2266586, 6)

(2266586, 6)

In [18]:
new_train = pd.merge(left = new_train, right = new_test, how = 'outer') # new_train과 합함

In [19]:
new_train.shape # (2525956, 6)

(2525956, 6)

# 기본 전처리
- 새로운 column 추가
   - question_number: 문제가 테스트 상에서 몇 번인지
   - test_cat: 시험의 대분류 category가 몇인지
   - test_id: 시험지 고유의 번호
   - test_month: Timestamp에 찍힌 달
   - test_day: Timestamp에 찍힌 일
   - test_hour: Timestamp에 찍힌 시간
- 날짜 데이터 변경
   - 2019년 12월 31일 데이터에 대해 2020년 1월 1일로 변경 후 test_year는 버림

In [20]:
new_train['question_number'] = new_train['assessmentItemID'].apply(lambda x: x[8:10]).map(int) # 문제 순서 번호
new_train['test_cat'] = new_train['testId'].apply(lambda x: x[2]) # 시험지 대분류
new_train['test_id'] = new_train['assessmentItemID'].apply(lambda x: x[4:7]) # 시험지 고유 번호
new_train["Timestamp"] = new_train["Timestamp"].astype("str") # timestamp 처리

In [21]:
new_train['test_year'] = new_train['Timestamp'].apply(lambda x: x[:4])
new_train['test_day'] = new_train['Timestamp'].apply(lambda x: x[8:10])
new_train['test_hour'] = new_train['Timestamp'].apply(lambda x: x[11:13])
new_train['test_month'] = new_train['Timestamp'].apply(lambda x: x[5:7])

In [22]:
# timestamp에서 2019년 데이터는 없앰
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_year'] = '2020'
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_month'] = '01'
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_day'] = '01'

In [23]:
new_train = new_train.drop(['test_year'], axis = 1)

# [전처리] 문항, 시험지, 태그의 평균 정답률#14

- new_train2 생성

In [24]:
# 문항 시험지 태그의 평균정답률
# 시험지의 평균 정답률
test_acc = new_train.groupby(['test_cat', 'test_id'])['answerCode'].mean().reset_index()
test_acc.columns = ['test_cat','test_id','test_acc']
new_train2 = pd.merge(left = new_train, right = test_acc, on = ['test_cat', 'test_id'], how = 'left')

# 문항의 평균 정답률
question_acc = new_train.groupby(['test_cat', 'test_id', 'question_number'])['answerCode'].mean().reset_index()
question_acc.columns = ['test_cat', 'test_id', 'question_number', 'question_acc']
new_train2 = pd.merge(left = new_train2, right = question_acc, on = ['test_cat', 'test_id', 'question_number'], how = 'left')

# 태그의 평균 정답률
tag_acc = new_train.groupby(['KnowledgeTag'])['answerCode'].mean().reset_index()
tag_acc.columns = ['KnowledgeTag', 'tag_acc']
new_train2 = pd.merge(left = new_train2, right = tag_acc, on = ['KnowledgeTag'], how = 'left')

In [25]:
# 시간별 정답률
hour_acc = new_train.groupby(['test_hour'])['answerCode'].mean().reset_index()
hour_acc.columns = ['test_hour', 'hour_acc']
new_train2 = pd.merge(left = new_train2, right = hour_acc, on = ['test_hour'], how = 'left')

# 월별 정답률
month_acc = new_train.groupby(['test_month'])['answerCode'].mean().reset_index()
month_acc.columns = ['test_month', 'month_acc']
new_train2 = pd.merge(left = new_train2, right = month_acc, on = ['test_month'], how = 'left')

# 누적 정답률
user_acc = new_train.groupby('userID')['answerCode'].mean().reset_index()
user_acc.columns = ['userID', 'user_acc']
new_train2 = pd.merge(left = new_train2, right = user_acc, on = 'userID', how = 'left')

- new_train3 생성

In [26]:
new_train3 = new_train2.copy()

# [전처리] 해당 시험/문제/태그를 몇 번째 푸는지 여부

In [27]:
# 문제를 몇 번 푸는지?
exp_question = new_train3.groupby(['test_cat', 'test_id', 'question_number'])['userID'].value_counts().reset_index(name='exp_question')
new_train3 = pd.merge(left = new_train3, right = exp_question, on = ['userID', 'test_cat', 'test_id', 'question_number'], how = 'left')

# 시험을 몇 번 푸는지?
fff = new_train3.groupby(['test_cat', 'test_id'])['question_number'].unique().reset_index()
fff['len'] = fff['question_number'].apply(len)

exp_test = new_train3.groupby(['test_cat', 'test_id'])['userID'].value_counts().reset_index(name='exp_test')
new_train3 = pd.merge(left = new_train3, right = exp_test, on = ['userID', 'test_cat', 'test_id'], how = 'left')
new_train3 = pd.merge(left = new_train3, right = fff, on = ['test_cat', 'test_id'], how = 'left')

In [28]:
# exp_test 정상적인 값으로 바꿈
new_train3['exp_test'] = new_train3.apply(lambda x: x['exp_test'] / x['len'], axis = 1)
new_train3.rename(columns = {'question_number_x':'question_number','question_number_y':'question_nums', 'len':'question_numslen'},inplace=True)
new_train3['exp_test'] = new_train3['exp_test'].astype(int)

In [29]:
# 태그를 몇 번 푸는지?
exp_tag = new_train3.groupby(['KnowledgeTag'])['userID'].value_counts().reset_index(name='exp_tag')
new_train3 = pd.merge(left = new_train3, right = exp_tag, on = ['userID', 'KnowledgeTag'], how = 'left')

# 정리
## train
- 원본
## new_train
- 다른 기법 없이 기존 columns만 정리하여 새로운 column 생성
- userID	
- answerCode	
- Timestamp	
- KnowledgeTag	
- question_number	
- test_cat	
- test_id	
- test_day	
- test_hour	
- test_month
## new_train2
- 새로운 기법(완료된 project 참조) 적용해 column 생성
- userID	
- answerCode	
- Timestamp	
- KnowledgeTag	
- question_number	
- test_cat	
- test_id	
- test_day	
- test_hour	
- test_month	
- test_acc	
- question_acc	
- tag_acc	
- hour_acc	
- month_acc	
- user_acc

## new_train3 (11/21 월요일)
- 태그, 문제, 테스트 중복 횟수 column 추가~
- userID: 유저 아이디
- answerCode: 정답 여부 1: 정답 0: 오답
- Timestamp: 문제를 풀기 시작한 시간
- KnowledgeTag: 문제 유형 태그
- test_cat: 시험의 대분류
- test_id: 시험의 고유 아이디
- question_number: 시험에서의 문제 번호
   - test_cat, test_id, question_number는 기존의 assessmentItemID를 분해한 것
- question_numslen: 시험에서 몇 개의 번호가 존재하는가 (1,2,3,5가 출제되었다면 question_numslen는 4)
- test_month: 시험을 본 달
- test_day: 시험을 본 일
- test_hour: 시험을 본 시간
   - test_month, test_day, test_hour는 Timestamp에서 의미가 있다고 판단되는 feature
   - 2019/12/31이 일부 있었으나, 2020/01/01로 바꾸어서 test_year가 의미 없어짐
- user_acc: 해당 유저의 전체 정답률
- test_acc: 해당 test의 정답률
- tag_acc: 해당 tag의 정답률
- question_acc: 해당 문제의 정답률
- month_acc: 이 달의 전체 유저의 정답률
- hour_acc: 이 시간의 전체 유저의 정답률
- exp_test: 해당 테스트를 경험한 수 (최소 1 최대 3)
- exp_tag: 해당 태그를 경험한 수
- exp_question: 해당 문제를 경험한 수(대체로 exp_test를 따라갈 것)

- new_train4 생성

In [30]:
new_train4 = new_train3.copy()

In [31]:
new_train4['post_Timestamp'] = new_train4.groupby(['userID', 'test_id', 'test_month', 'test_day'])['Timestamp'].shift(-1)

#### 수경 EDA에서 Elapsed time 가져옴

In [32]:
# elapsed time: 문제 푸는데 소요 된 시간
new_train4['Timestamp'] = pd.to_datetime(new_train4['Timestamp'])
new_train4['elapsed_time'] = new_train4['Timestamp'].astype(int)

tmp = list(new_train4['elapsed_time'].diff() // 10**9)
tmp.append(99999)
new_train4['elapsed_time'] = tmp[1:]

# user 또는 시험지가 바뀔 때 값이 잘못 들어가서 그 경우를 -1로 바꿔줌
# -> 이 경우에는 user가 같은 시험지를 풀었을 때 값을 고려 못 해줌
new_train4['Timestamp'] = new_train4['Timestamp'].astype(str)
tmp_dict = new_train4.groupby(['userID', 'testId'])['Timestamp'].max().to_frame().reset_index()
tmp_dict['rev_time'] = -1
tmp_dict = tmp_dict.set_index(['userID', 'testId', 'Timestamp']).to_dict()
tmp_dict = tmp_dict['rev_time']
new_train4['tmp'] = tuple(zip(new_train4.userID, new_train4.testId, new_train4.Timestamp))
new_train4['tmp'] = new_train4['tmp'].map(tmp_dict)
new_train4['tmp'] = new_train4['tmp'].fillna(new_train4['elapsed_time'])
new_train4['elapsed_time'] = new_train4['tmp']
new_train4.drop(['tmp'], axis=1, inplace=True)

In [33]:
# 연속해서 동일한 시험지를 푼 사람들을 -1 붙여 줌

tmp = 1
same_test = []
for i in tqdm(range(1,len(new_train4))):
    cur_test = new_train4['testId'][i]
    prev_test = new_train4['testId'][i-1]
    if cur_test == prev_test:
        tmp += 1
        if tmp > new_train4['question_numslen'][i]:
            same_test.extend([i-1])
            tmp = 1
    else:
        tmp = 1
        
new_train4.loc[same_test, 'elapsed_time'] = -1

100%|██████████| 2525955/2525955 [00:54<00:00, 46746.25it/s]


# 여기까지 11월 22일

#### Elapsed time에 문제가 되는 것들
- 우선! 아래 문제가 되는 것들은 NaN으로 두고, 모델에서 처리하는 것을 목표로 함
- 각 문제는 최소 45회 풀렸기 때문에 median 처리를 기본으로 함
- 일부 정상치가 이상치로 판단된다고 하더라도, 후에 적절히 채워 넣을 것이므로 상관 없다는 판단(데정연)
   - 유저가 바뀐다면? (처리 완료 post_Timestamp: NaN)
   - 다른 테스트를 푼다면? (처리 완료 post_Timestamp: NaN)
   - 같은 테스트를 시간 간격 없이 푼다면? (처리완료 elapsed_time: -1.0 [for문 돌려서 만약 numslen을 초과해버리면 -1 넣음] >> 다른 테스트를 풀고 와서 또 푸는 경우를 못 잡음
   - 같은 테스트를 시간 간격(day의 변화)을 두고 푼다면? 다른 걸 풀고 다른 날 또 온다면? (처리완료 post_Timestamp: NaN) [day 변화로 판단 >> 물론 정상치 1000개 정도가 -1로 처리됨]
      - 그럼 같은 날 또 온다면? 아마 이상치처럼 상당히 긴 시간으로 처리될 것 같다? -> threshold 적용 예정
   - 현재 문제 시작 시간과 다음 문제 시작 시간이 같다면? (처리완료 elapsed_time: 0)
   - 대부분 마지막 문제로 할당되는 문제들은 elapsed_time median도 -1.0으로 찍힘 (처리완료, 그거 제외하고 median 냄, 만약 다 지워진다면 -1로 채움)
   - 정상 데이터라도 딴 짓 하는 애들 때문에 엄청 큰 값이 잡힌다면? 어케 처리함? -> threshold 적용 예정
- 사실 결국 정상치든, 이상치든 시간에 대한 문제인데 그럼 Threshold를 몇으로 설정하여 NaN을 잡는가가 문제

In [34]:
new_train4.loc[new_train4['Timestamp'] == new_train4['post_Timestamp'], 'elapsed_time'] = 0 # Timestamp	post_Timestamp가 같다면, elapsed_time은 0
new_train4.loc[new_train4['post_Timestamp'].isna(), 'elapsed_time'] = -1.0 # post_Timestamp이 NaN이라면 (유저/테스트/날짜 바뀜) elapsed_time은 -1

- new_train5 생성

In [35]:
new_train5 = new_train4.copy() # nan 제거 후 median mean 구하기 위한 data frame

In [36]:
new_train5['elapsed_time'] = new_train5['elapsed_time'].replace(0, np.NaN)
new_train5['elapsed_time'] = new_train5['elapsed_time'].replace(-1, np.NaN)
new_train5 = new_train5.dropna(axis = 0)

In [37]:
# 모든 유저가 각 문제를 푸는 데 걸리는 시간의 중간값 question_time_median
question_time_median2 = new_train5.groupby('assessmentItemID')['elapsed_time'].median().reset_index()
question_time_median2.rename(columns = {'elapsed_time':'time_question_median'}, inplace = True)
new_train4 = pd.merge(left=new_train4, right= question_time_median2, on = 'assessmentItemID', how = 'left')

In [38]:
# 해당 유저가 모든 문제에 대해 푸는 데 걸리는 시간의 중간값 
user_time_median2 = new_train5.groupby('userID')['elapsed_time'].median().reset_index()
user_time_median2.rename(columns = {'elapsed_time':'time_user_median'}, inplace = True)
new_train4 = pd.merge(left=new_train4, right= user_time_median2, on = 'userID', how = 'left')

In [39]:
# 해당 유저가 모든 문제에 대해 푸는 데 걸리는 시간의 평균값 
user_time_mean2 = new_train5.groupby('userID')['elapsed_time'].mean().reset_index()
user_time_mean2.rename(columns = {'elapsed_time':'time_user_mean'}, inplace = True)
new_train4 = pd.merge(left=new_train4, right= user_time_mean2, on = 'userID', how = 'left')

In [40]:
# NaN인 time_question_median을 -1로 변경
new_train4['time_question_median'] = new_train4['time_question_median'].replace(np.NaN, -1)

In [41]:
new_train4['Timestamp'] = pd.to_datetime(new_train4['Timestamp'])
new_train4['week_num'] = new_train4['Timestamp'].apply(lambda x: x.weekday())

In [42]:
new_train4 = new_train4[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'post_Timestamp','week_num', 'elapsed_time', 'time_question_median',
       'time_user_median', 'time_user_mean', 'KnowledgeTag', 'test_cat',
       'test_id', 'question_number', 'question_numslen', 'test_month',
       'test_day', 'test_hour', 'user_acc', 'test_acc', 'tag_acc',
       'question_acc', 'month_acc', 'hour_acc', 'exp_test', 'exp_tag',
       'exp_question']]

## new_train3 (11/23 수요일)
- userID: (기존) 유저 아이디
- assessmentItemID: (기존)  문제 고유 코드
- testId: (기존) 시험 고유 코드
- answerCode: (기존) 정답 여부 1: 정답 0: 오답
- Timestamp: (기존) 문제를 풀기 시작한 시간
- post_Timestamp: 다음 문제를 풀기 시작한 시간, 마지막 문제의 경우 NaN
- week_num: Timestamp의 요일, 1:월요일부터 시작
- elapsed_time: 현재 문제 시작부터 다음 문제 시작까지의 시간
- 시간에 대한 추가적인 feature (모두 NaN(-1.0)은 제거하고 계산됨)
   - time_question_median: 해당 문제에 대해 모든 유저가 소요한 시간의 중앙값 -> NaN 값은 -1.0으로 채워짐
      - 이상치 NaN을 제거하고 계산하다보니, 해당 문제를 아무도 풀지 않은 것처럼 되어 NaN이 생긴 것
   - time_user_median: 해당 유저가 푼 모든 문제에 대해 소요한 시간의 중앙값
   - time_user_mean: 해당 유저가 푼 모든 문제에 대해 소요한 시간의 평균값
- KnowledgeTag: (기존) 문제 유형 태그
- test_cat: 시험의 대분류
- test_id: 시험의 고유 아이디
- question_number: 시험에서의 문제 번호
   - test_cat, test_id, question_number는 기존의 assessmentItemID를 분해한 것
- question_numslen: 시험에서 몇 개의 번호가 존재하는가 (1,2,3,5가 출제되었다면 question_numslen는 4)
- test_month: 시험을 본 달
- test_day: 시험을 본 일
- test_hour: 시험을 본 시간
   - test_month, test_day, test_hour는 Timestamp에서 의미가 있다고 판단되는 feature
   - 2019/12/31이 일부 있었으나, 2020/01/01로 바꾸어서 test_year가 의미 없어짐
- user_acc: 해당 유저의 전체 정답률
- test_acc: 해당 test의 정답률
- tag_acc: 해당 tag의 정답률
- question_acc: 해당 문제의 정답률
- month_acc: 이 달의 전체 유저의 정답률
- hour_acc: 이 시간의 전체 유저의 정답률
- exp_test: 해당 테스트를 경험한 수 (최소 1 최대 3)
- exp_tag: 해당 태그를 경험한 수
- exp_question: 해당 문제를 경험한 수(대체로 exp_test를 따라갈 것)

In [43]:
new_train4

,userID,assessmentItemID,testId,answerCode,Timestamp,post_Timestamp,week_num,elapsed_time,time_question_median,time_user_median,...,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,2020-03-24 00:17:14,1,3.0,6.0,20.0,...,00,0.630872,0.952667,0.957333,0.984000,0.681989,0.650799,1,1,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,2020-03-24 00:17:22,1,8.0,14.0,20.0,...,00,0.630872,0.952667,0.917067,0.968000,0.681989,0.650799,1,5,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,2020-03-24 00:17:29,1,7.0,10.0,20.0,...,00,0.630872,0.952667,0.917067,0.916000,0.681989,0.650799,1,5,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,2020-03-24 00:17:36,1,7.0,11.0,20.0,...,00,0.630872,0.952667,0.917067,0.972000,0.681989,0.650799,1,5,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,2020-03-24 00:17:47,1,11.0,20.5,20.0,...,00,0.630872,0.952667,0.917067,0.948000,0.681989,0.650799,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7439,A040197006,A040000197,1,2020-08-21 07:39:45,NaN,4,-1.0,12.5,28.0,...,07,0.733333,0.773081,0.765016,0.953020,0.657521,0.642030,1,6,1
2525952,7439,A040130001,A040000130,0,2020-10-14 23:07:23,2020-10-14 23:07:41,2,18.0,31.5,28.0,...,23,0.733333,0.640427,0.638385,0.436667,0.670603,0.659126,0,2,1
2525953,7439,A040130002,A040000130,1,2020-10-14 23:07:41,2020-10-14 23:08:02,2,21.0,16.0,28.0,...,23,0.733333,0.640427,0.638385,0.483333,0.670603,0.659126,0,2,1
2525954,7439,A040130003,A040000130,1,2020-10-14 23:08:02,2020-10-14 23:09:31,2,89.0,27.0,28.0,...,23,0.733333,0.640427,0.765442,0.846667,0.670603,0.659126,0,2,1


In [44]:
# new_train4.to_csv("train_v1.1.csv", index = False)

# 정답 cumsum 추가

In [45]:
new_train4['shift'] = new_train4.groupby('userID')['answerCode'].shift().fillna(0)
new_train4['ans_cumsum'] = new_train4.groupby('userID')['shift'].cumsum()
new_train4 = new_train4.drop('shift', axis=1)


## 연속 정답 추가

In [46]:
new_train4["continuous_user"] = new_train4.groupby("userID")["answerCode"]\
    .apply(lambda x: x * (x.groupby((x != x.shift()).cumsum()).cumcount() + 1))

new_train4["continuous_test"] = new_train4.groupby(["userID", "testId"])["answerCode"]\
    .apply(lambda x: x * (x.groupby((x != x.shift()).cumsum()).cumcount() + 1))

/tmp/ipykernel_46665/3091558490.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  new_train4["continuous_user"] = new_train4.groupby("userID")["answerCode"]\
/tmp/ipykernel_46665/3091558490.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  new_train4["continuous_test"] = new_train4.groupby([

In [47]:
# new_train4.to_csv("train_v1.1.csv", index = False)

In [48]:
pd.set_option('display.max_columns', None)

In [49]:
new_train4

,userID,assessmentItemID,testId,answerCode,Timestamp,post_Timestamp,week_num,elapsed_time,time_question_median,time_user_median,time_user_mean,KnowledgeTag,test_cat,test_id,question_number,question_numslen,test_month,test_day,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question,ans_cumsum,continuous_user,continuous_test
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,2020-03-24 00:17:14,1,3.0,6.0,20.0,51.151420,7224,6,001,1,6,03,24,00,0.630872,0.952667,0.957333,0.984000,0.681989,0.650799,1,1,1,0.0,1,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,2020-03-24 00:17:22,1,8.0,14.0,20.0,51.151420,7225,6,001,2,6,03,24,00,0.630872,0.952667,0.917067,0.968000,0.681989,0.650799,1,5,1,1.0,2,2
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,2020-03-24 00:17:29,1,7.0,10.0,20.0,51.151420,7225,6,001,3,6,03,24,00,0.630872,0.952667,0.917067,0.916000,0.681989,0.650799,1,5,1,2.0,3,3
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,2020-03-24 00:17:36,1,7.0,11.0,20.0,51.151420,7225,6,001,4,6,03,24,00,0.630872,0.952667,0.917067,0.972000,0.681989,0.650799,1,5,1,3.0,4,4
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,2020-03-24 00:17:47,1,11.0,20.5,20.0,51.151420,7225,6,001,5,6,03,24,00,0.630872,0.952667,0.917067,0.948000,0.681989,0.650799,1,5,1,4.0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7439,A040197006,A040000197,1,2020-08-21 07:39:45,NaN,4,-1.0,12.5,28.0,41.583333,2132,4,197,6,6,08,21,07,0.733333,0.773081,0.765016,0.953020,0.657521,0.642030,1,6,1,7.0,1,1
2525952,7439,A040130001,A040000130,0,2020-10-14 23:07:23,2020-10-14 23:07:41,2,18.0,31.5,28.0,41.583333,8832,4,130,1,5,10,14,23,0.733333,0.640427,0.638385,0.436667,0.670603,0.659126,0,2,1,8.0,0,0
2525953,7439,A040130002,A040000130,1,2020-10-14 23:07:41,2020-10-14 23:08:02,2,21.0,16.0,28.0,41.583333,8832,4,130,2,5,10,14,23,0.733333,0.640427,0.638385,0.483333,0.670603,0.659126,0,2,1,8.0,1,1
2525954,7439,A040130003,A040000130,1,2020-10-14 23:08:02,2020-10-14 23:09:31,2,89.0,27.0,28.0,41.583333,8244,4,130,3,5,10,14,23,0.733333,0.640427,0.765442,0.846667,0.670603,0.659126,0,2,1,9.0,2,2


In [50]:
new_train4.to_csv("traintest_v1.1.csv", index = False)

# test_data에다가 위에서 구한 feature를 붙여 줌
- 이 아래는 11월 23일 밤에 한 실험이 포함되어 있습니다.

In [4]:
new_train4 = pd.read_csv('./traintest_v1.1.csv')

In [33]:
def get_now_elapsed(df):
    
    diff = df.loc[:, ['userID','Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())
    df['now_elapsed'] = diff
    df['now_elapsed'] = df['now_elapsed'].apply(lambda x : x if x < 650 and x >=0 else 0)
    df['now_elapsed'] = df['now_elapsed']

    return df


In [35]:
train["Timestamp"] = pd.to_datetime(train["Timestamp"])
test_train = get_now_elapsed(train)